In [ ]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
! pip install -U accelerate


In [ ]:
from transformers import TrainingArguments, Trainer
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline
import random
from datasets import Dataset

#  Fine-tuning a pretrained model



## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
from datasets import load_dataset

# insert your code here
dataset=load_dataset('yelp_polarity')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for yelp_polarity contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/yelp_polarity
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.war

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})


In [ ]:
import random
print(f' train : {dataset["train"]}')
print(f' test : {dataset["test"]} ')
print(f' An example row: {dataset["train"]["text"][0]}')
print(f'class 0 count in train: {dataset["train"]["label"].count(0)}')
print(f'class 1 count in train: {dataset["train"]["label"].count(1)}')
print(f'class 0 count in test: {dataset["test"]["label"].count(0)}')
print(f'class 1 count in test: {dataset["test"]["label"].count(1)}')

##GENERATE 300 SAMPLES FROM TRAIN AND TEST RANDOMLY WITH BALANCED CLASSES, 150 FROM EACH AND SHUFFLE AFTER TO KEEP THE STOHASTICITY
train_data=dataset['train']
test_data=dataset['test']
L1=[]
L2=[]
for cl in [0,1]:
  train_ind =[i for i,label in enumerate(train_data['label']) if label==cl]
  test_ind =[i for i,label in enumerate(test_data['label']) if label==cl]
  random.shuffle(train_ind)
  random.shuffle(test_ind)
  sel_train_ind=train_ind[:150]
  sel_test_ind=test_ind[:150]
  L1.extend(sel_train_ind)
  L2.extend(sel_test_ind)

random.shuffle(L1)
random.shuffle(L2)

new_data_train={'text': [train_data['text'][i] for i in L1], 'label': [train_data['label'][i] for i in L1]}
new_data_test={'text': [test_data['text'][i] for i in L2], 'label': [test_data['label'][i] for i in L2]}



 train : Dataset({
    features: ['text', 'label'],
    num_rows: 560000
})
 test : Dataset({
    features: ['text', 'label'],
    num_rows: 38000
}) 
 An example row: Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.
class 0 count in train: 280000
class 1 count in train: 280000
class 0 count in test: 19000
class 1 count in test: 19000


# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.



Επιλέξα το roberta-case από το Huggingface για το συγκεκριμένο ερώτημα

In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here
tokenized_train=tokenize_function(new_data_train)
tokenized_test=tokenize_function(new_data_test)

In [ ]:
print(tokenized_train.keys())
print(tokenized_test.keys())

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])


Επέλεξα να χρησιμοποιήσω την κλάση Trainer από το Huggingface για το fine-tuning του μοντέλου. Αυτή η προσέγγιση με βοήθησε να απλοποιήσω σημαντικά τη διαδικασία εκπαίδευσης, καθώς μου επέτρεψε να αποφύγω την ανάγκη δημιουργίας ενός custom training loop. Επιπλέον, παρόλο που η Trainer δεν ενσωματώνει αυτόματα μηχανισμούς αξιολόγησης της απόδοσης, ενσωμάτωσα μια επιπλέον συνάρτηση για τον υπολογισμό του accuracy σε κάθε εποχή, κάτι που συνέβαλε στην καλύτερη παρακολούθηση της προόδου του μοντέλου κατά τη διάρκεια της εκπαίδευσης.

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer, TrainingArguments, AdamW, get_scheduler

from datasets import Dataset

train_dataset = Dataset.from_dict({
    'input_ids': tokenized_train['input_ids'],
    'attention_mask': tokenized_train['attention_mask'],
    'label': new_data_train['label']
})

test_dataset= Dataset.from_dict({
    'input_ids': tokenized_test['input_ids'],
    'attention_mask': tokenized_test['attention_mask'],
    'label': new_data_test['label']
})

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=8,num_train_epochs=3)
params=model.parameters()

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


#optimizer
optimizer = AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01)

# scheduler
num_training_steps = len(trainer.get_train_dataloader()) * args.num_train_epochs
scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

trainer.optimizer = optimizer
trainer.lr_scheduler = scheduler


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
!nvidia-smi
trained_model=trainer.train()


Παρακάτω παρουσιάζεται ο πίνακας με τις υπερπαραμέτρους που δοκιμάστηκαν. Έγινε χρήση του ADAM optimizer και πειραματιστήκαμε με τις learning rate και weight_decay υπερπαραμέτρους ενώ για τον scheduler δοκιμάσαμε linear, constant, cosine. Τελος για batch_size δοκιμάσαμε τιμές μόνο 8,16 καθώς για 32 πετούσε error στην μνήμη. Οι πιο σημαντικές υπερπαραμέτροι βάση των αποτελέσματων φαίνεται να είναι το είδος του scheduler  και το learning rate. Για constant scheduler έχουμε τα χειρότερα accuracies ενώ για cosine και linear τα αποτελέσματα είναι ενθαρρυντικά. To learning rate πρέπει να είναι αρκετά μικρό καθώς κάνουμε fine tuning και ακόμα και μικρές αλλαγές δηλαδή από 5e-5 σε 5e-4 ή από 5e-5 σε 5e-6 ρίχνουν δραματικά το accuracy. Για το batch size  δεν είδαμε μεγάλη διαφορά. Πάντως για τον constant scheduler η χρήση μικρότερου batch size έφερε πολύ καλύτερα αποτελέσματα ειδικά σε συνδιασμό με υψηλότερο learning rate. Το weight_decay επίσης δεν έδειξε να επηρεάζει σημαντικά τα αποτελέσματα.  Ο καλύτερος συνδιασμός υπερπαραμέτρων γενικά είναι αυτός στην πρώτη γραμμή του πίνακα με accuracy 94.66%

| Scheduler | Learning Rate | Batch Size | Weight Decay | Accuracy |
|-----------|---------------|------------|--------------|----------|
| Linear    | 5e-5          | 16         | 0.001        | 0.946667 |
| Linear    | 5e-5          | 16         | 0.01         | 0.940000 |
| Linear    | 5e-5          | 8          | 0.01         | 0.933333 |
| Linear    | 5e-5          | 8          | 0.001        | 0.930000 |
| Linear    | 1e-4          | 8          | 0.01         | 0.930000 |
| Linear    | 1e-4          | 8          | 0.001        | 0.910000 |
| Linear    | 1e-4          | 16         | 0.01         | 0.940000 |
| Linear    | 1e-4          | 16         | 0.001        | 0.903333 |
| Constant  | 5e-5          | 16         | 0.01         | 0.740000 |
| Constant  | 1e-4          | 16         | 0.01         | 0.816667 |
| Constant  | 5e-5          | 8          | 0.01         | 0.816667 |
| Constant  | 1e-4          | 8          | 0.01         | 0.926667 |
| Cosine    | 5e-5          | 16         | 0.01         | 0.883333 |
| Cosine    | 1e-4          | 16         | 0.01         | 0.913333 |
| Cosine    | 5e-5          | 8          | 0.01         | 0.923333 |


Στο συγκεκριμένο μέρος αξιοποιούνται τα fine-tuned μοντέλα για την αντιμετώπιση νέων γλωσσικών tasks, εφαρμόζοντας αρχές του transfer learning. Ο στόχος είναι να μετατραπούν σύνθετα γλωσσικά προβλήματα σε πιο παραδοσιακά tasks, όπως το text classification, το natural language inference και το question answering.

Για παράδειγμα, τα μοντέλα που έχουν fine-tuned για text classification μπορούν να εφαρμοστούν σε tasks όπως:

Αξιολόγηση Παραφράσεων: Προσδιορίζουν εάν δύο προτάσεις είναι παραφράσεις η μία της άλλης.

Εννοιολογική Συνεπαγωγή: Εξετάζουν αν μια πρόταση συνεπάγεται, είναι ουδέτερη ή αντιφατική με μια άλλη.

Αξιολόγηση Γραμματικής Ορθότητας: Κατηγοριοποιούν προτάσεις ως γραμματικά ορθές ή μη.

Η προσέγγιση αυτή εκμεταλλεύεται την εμπειρία που έχει αποκτηθεί από προηγούμενα tasks, επιτρέποντας την εφαρμογή της σε νέα και πιο περίπλοκα προβλήματα, χωρίς να απαιτείται από την αρχή εκπαίδευση ενός νέου μοντέλου.

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέγεται ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)
from datasets import load_dataset

dataset=load_dataset('piqa')

Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 16113
    })
    test: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 3084
    })
    validation: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 1838
    })
})

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import random

dataset = load_dataset('piqa')

train_data = dataset['train']
eval_data = dataset['validation']

L1 = [i for i, label in enumerate(train_data['label'])]
L2 = [i for i, label in enumerate(eval_data['label'])]

random.shuffle(L1)
random.shuffle(L2)

train_dataset = Dataset.from_dict({
    'goal': [train_data['goal'][i] for i in L1[:300]],
    'sol1': [train_data['sol1'][i] for i in L1[:300]],
    'sol2': [train_data['sol2'][i] for i in L1[:300]],
    'label': [train_data['label'][i] for i in L1[:300]]
})

validation_dataset = Dataset.from_dict({
    'goal': [eval_data['goal'][i] for i in L2[:300]],
    'sol1': [eval_data['sol1'][i] for i in L2[:300]],
    'sol2': [eval_data['sol2'][i] for i in L2[:300]],
    'label': [eval_data['label'][i] for i in L2[:300]]
})

test_dataset = dataset['test']

merged_dataset = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

dataset = merged_dataset



In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 300
    })
    validation: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 300
    })
    test: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 3084
    })
})

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.



Μοντέλα κλασσικά όπως τα BERT,DISTILBERT,ROBERTA κτλπ έχουν τα output layers τυχαία αρχικοποιημένα και χρειάζονται fine tuning για να λειτουργήσουν. Επομένως εδω θα χρησιμοποιήσουμε μοντέλα του huggingface τα οποία είναι fine tuned για να κάνουν multiplechoice tasks.

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice
import torch

def val_func(dataset,model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMultipleChoice.from_pretrained(model_name)
    sum=0
    for i in range(len(dataset['goal'])):

        prompt  =dataset['goal'][i]
        choice1 =dataset['sol1'][i]
        choice2 =dataset['sol2'][i]
        label   =dataset['label'][i]
        labels = torch.tensor(label).unsqueeze(0)


        inputs = tokenizer([prompt, prompt], [choice1, choice2], return_tensors="pt", padding=True)
        outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)

        logits = outputs.logits
        predicted_class = logits.argmax().item()
        if predicted_class==label:
            sum+=1
    return sum/300

model_name=["LIAMF-USP/roberta-large-finetuned-race",
            'sledz08/finetuned-bert-piqa',
            'namban4123/Deberta_for_multiple_choice',
            'Riiid/kda-mpnet-base-race',
            'LIAMF-USP/aristo-roberta'
]
accuracy_res={}
for i in model_name:
  accuracy_res[i]=val_func(validation_dataset,i)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Παρακάτω βλέπουμε τα αποτελέσματα από τα μοντέλα μας, γενικά τα αποτελέσματα ειναι αρκετά καλά, έχουμε αρκετά υψηλότερα accuracies σε όλα απ'ότι θα είχαμε αν επιλέγαμε τυχαία δηλαδή 50% και βρίσκονται περίπου στο 60-70%. Το ότι δεν έχουμε μεγαλύτερα accuracies όπως στο προηγούμενο ερώτημα οφείλεται στο οτι τα μοντέλα δεν έχουν γίνει fine tuned στο δικό μας (piqa) dataset πέρα από το sledz08/finetuned-bert-piqa το οποίο περιέργως αν και έγινε fine tuned στο piqa δεν έχει τόσο καλά αποτελέσματα. Το καλύτερο μοντέλο μας είναι το namban4123/Deberta_for_multiple_choice με accuracy 74% το οποίο έχει γίνει fine tuned σε δεδομένα που είχαν 5 επιλογές για κάθε ερώτηση.

In [ ]:
accuracy_res

{'LIAMF-USP/roberta-large-finetuned-race': 0.6533333333333333,
 'sledz08/finetuned-bert-piqa': 0.6266666666666667,
 'namban4123/Deberta_for_multiple_choice': 0.7433333333333333,
 'Riiid/kda-mpnet-base-race': 0.5633333333333334,
 'LIAMF-USP/aristo-roberta': 0.6466666666666666}

 Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.4048847

# Περιγραφή της Προσέγγισης για τον Υπολογισμό του Accuracy

Σε αυτήν την ενότητα, περιγράφεται η μέθοδος με την οποία αξιολογείται το accuracy στην εύρεση των "βελτιστα σωστών" απαντήσεων, χρησιμοποιώντας δεδομένα από το dataset "Truthful QA generation". Η διαδικασία έχει ως εξής:

- **Επιλογή Δειγμάτων:**  
  Επιλέγουμε 100 δείγματα από το dataset, εξασφαλίζοντας ότι κάθε δείγμα περιέχει τουλάχιστον δύο "correct answer".

- **Ορισμός Υποψηφίων Απαντήσεων:**  
  Για κάθε δείγμα ορίζονται τέσσερις υποψήφιες απαντήσεις:  
  1. Best answer  
  2. 1η Correct answer  
  3. 2η Correct answer  
  4. Incorrect answer

- **Μοντέλο Πολλαπλής Επιλογής:**  
  Χρησιμοποιούμε ένα μοντέλο πολλαπλής επιλογής, το οποίο λαμβάνει ως είσοδο την ερώτηση μαζί με τις 4 υποψήφιες απαντήσεις και επιστρέφει τον δείκτη της προβλεπόμενης απάντησης.

- **Semantic Similarity:**  
  Για να αξιολογήσουμε την ορθότητα, ειδικά όταν το μοντέλο επιλέγει μία από τις δύο correct απαντήσεις, λαμβάνουμε διανυσματικές αναπαραστάσεις (embeddings) για την best answer και για την επιλεγείσα correct απάντηση, χρησιμοποιώντας τουλάχιστον 6 διαφορετικά μοντέλα semantic similarity.  
  Ο στόχος είναι να μετρήσουμε το cosine similarity μεταξύ των embeddings της best answer και της επιλεγείσας correct απάντησης.

- **Κριτήριο Αξιολόγησης:**  
  Αν το cosine similarity υπερβαίνει το καθορισμένο όριο (threshold) των 0.95 για τουλάχιστον ένα από τα semantic similarity μοντέλα, τότε η απάντηση θεωρείται "βελτιστα σωστή". Αυτό το αποτέλεσμα συνυπολογίζεται θετικά στον τελικό υπολογισμό του accuracy.

- **Συνολική Αξιολόγηση:**  
  Το τελικό accuracy υπολογίζεται ως το ποσοστό των δειγμάτων για τα οποία ο συνδυασμός του μοντέλου πολλαπλής επιλογής και του ελέγχου semantic similarity οδήγησε σε βέλτιστα σωστή απάντηση.

Αυτή η προσέγγιση επιτρέπει μια πιο λεπτομερή αξιολόγηση της επίδοσης του μοντέλου, λαμβάνοντας υπόψη τόσο την πρόβλεψη του μοντέλου όσο και τη νοηματική ομοιότητα μεταξύ των απαντήσεων.


In [ ]:
# insert your code here (load dataset)
dataset = load_dataset('truthful_qa','generation')
dataset=dataset['validation'].filter(lambda x: len(x['correct_answers'])>=2)
dataset=dataset.select(range(100))

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Filter:   0%|          | 0/817 [00:00<?, ? examples/s]

In [ ]:
# insert your code here (load models for semantic similarity and QA)
model_name=["LIAMF-USP/roberta-large-finetuned-race",
            'sledz08/finetuned-bert-piqa',
            'namban4123/Deberta_for_multiple_choice',
            'Riiid/kda-mpnet-base-race',
            'LIAMF-USP/aristo-roberta'
]

sent_transf=['sentence-transformers/all-mpnet-base-v2',
             'sentence-transformers/all-MiniLM-L6-v2',
             'sentence-transformers/msmarco-MiniLM-L-12-v3',
             'jhgan/ko-sroberta-multitask',
             'sentence-transformers/stsb-roberta-base',
             'sentence-transformers/bert-base-nli-mean-tokens']

In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)
def sent_sim(dataset,sent_transf,model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMultipleChoice.from_pretrained(model_name)
    sent_transf=SentenceTransformer(sent_transf)
    sum=0
    for i in range(len(dataset['question'])):
        question       =dataset['question'][i]
        best_answer    =dataset['best_answer'][i]
        correct_answer1=dataset['correct_answers'][i][0]
        correct_answer2=dataset['correct_answers'][i][1]
        wrong_answer   =dataset['incorrect_answers'][i][0]


        label=torch.tensor(0).unsqueeze(0)
        inputs = tokenizer([question,question,question,question],[best_answer,correct_answer1,correct_answer2,wrong_answer], return_tensors="pt", padding=True)
        outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=label)
        logits = outputs.logits
        predicted_class = logits.argmax().item()

        if predicted_class==0:
            sum+=1
        elif predicted_class==1:
            correct_answer1=sent_transf.encode(correct_answer1)
            best_answer    =sent_transf.encode(best_answer)
            if get_cosine_similarity(correct_answer1,best_answer)>0.95:
                sum+=1
        elif predicted_class==2:
            correct_answer2=sent_transf.encode(correct_answer2)
            best_answer    =sent_transf.encode(best_answer)
            if get_cosine_similarity(correct_answer2,best_answer)>0.95:
                 sum+=1
    return sum/100

In [ ]:
acc={}
for i in model_name:
  for j in sent_transf:
    acc[(i,j)]=sent_sim(dataset,j,i)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

  warnings.warn(

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

  warnings.warn(

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

  warnings.warn(

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be remo

Εδώ ένα τυχαίο μοντέλο θα είχε 25% accuracy( στο περιπου). Εμάς οι περισσότεροι συνδιασμοί μοντέλων multiplechoice μοντέλων και sentence transformers είναι περίπου στο 33-41% με καλύτερο συνδιασμό να έχει accuracy 41%. Εδώ είναι αρκετά δύσκολο ένα μοντέλο που δεν έχει γίνει fine tuned στο dataset μας να τα πάει αρκετά καλύτερα διότι οι ερωτήσεις είναι εγκυκλοπεδικής φύσης και όχι ερωτήσεις που μπορούν να απαντηθούν με κοινή λογική την οποία μοντέλα μπορούν να αποκτήσουν και από άλλα datasets. Επίσης εδώ για κάθε ερώτηση έχουμε 4 πιθανές απαντήσεις για κάθε ερώτηση. Ένας ακόμα λόγος για τα όχι τόσο καλά αποτελέσματα είναι οτι το 95% είναι πολυ αυστηρό κατώφλι ομοιότητας, επομένως ακυρώνουμε πολλές σωστές επιλογές.

In [ ]:
acc

{('LIAMF-USP/roberta-large-finetuned-race',
  'sentence-transformers/all-mpnet-base-v2'): 0.41,
 ('LIAMF-USP/roberta-large-finetuned-race',
  'sentence-transformers/all-MiniLM-L6-v2'): 0.41,
 ('LIAMF-USP/roberta-large-finetuned-race',
  'sentence-transformers/msmarco-MiniLM-L-12-v3'): 0.4,
 ('LIAMF-USP/roberta-large-finetuned-race',
  'jhgan/ko-sroberta-multitask'): 0.44,
 ('LIAMF-USP/roberta-large-finetuned-race',
  'sentence-transformers/stsb-roberta-base'): 0.41,
 ('LIAMF-USP/roberta-large-finetuned-race',
  'sentence-transformers/bert-base-nli-mean-tokens'): 0.44,
 ('sledz08/finetuned-bert-piqa',
  'sentence-transformers/all-mpnet-base-v2'): 0.41,
 ('sledz08/finetuned-bert-piqa',
  'sentence-transformers/all-MiniLM-L6-v2'): 0.39,
 ('sledz08/finetuned-bert-piqa',
  'sentence-transformers/msmarco-MiniLM-L-12-v3'): 0.4,
 ('sledz08/finetuned-bert-piqa', 'jhgan/ko-sroberta-multitask'): 0.41,
 ('sledz08/finetuned-bert-piqa',
  'sentence-transformers/stsb-roberta-base'): 0.38,
 ('sledz08/

| Model Pair                                                                                     | Accuracy |
|------------------------------------------------------------------------------------------------|----------|
| LIAMF-USP/roberta-large-finetuned-race, sentence-transformers/all-mpnet-base-v2               | 0.41     |
| LIAMF-USP/roberta-large-finetuned-race, sentence-transformers/all-MiniLM-L6-v2                | 0.41     |
| LIAMF-USP/roberta-large-finetuned-race, sentence-transformers/msmarco-MiniLM-L-12-v3          | 0.40     |
| LIAMF-USP/roberta-large-finetuned-race, jhgan/ko-sroberta-multitask                           | 0.44     |
| LIAMF-USP/roberta-large-finetuned-race, sentence-transformers/stsb-roberta-base               | 0.41     |
| LIAMF-USP/roberta-large-finetuned-race, sentence-transformers/bert-base-nli-mean-tokens       | 0.44     |
| sledz08/finetuned-bert-piqa, sentence-transformers/all-mpnet-base-v2                          | 0.41     |
| sledz08/finetuned-bert-piqa, sentence-transformers/all-MiniLM-L6-v2                           | 0.39     |
| sledz08/finetuned-bert-piqa, sentence-transformers/msmarco-MiniLM-L-12-v3                     | 0.40     |
| sledz08/finetuned-bert-piqa, jhgan/ko-sroberta-multitask                                      | 0.41     |
| sledz08/finetuned-bert-piqa, sentence-transformers/stsb-roberta-base                          | 0.38     |
| sledz08/finetuned-bert-piqa, sentence-transformers/bert-base-nli-mean-tokens                  | 0.42     |
| namban4123/Deberta_for_multiple_choice, sentence-transformers/all-mpnet-base-v2               | 0.31     |
| namban4123/Deberta_for_multiple_choice, sentence-transformers/all-MiniLM-L6-v2                | 0.30     |
| namban4123/Deberta_for_multiple_choice, sentence-transformers/msmarco-MiniLM-L-12-v3          | 0.30     |
| namban4123/Deberta_for_multiple_choice, jhgan/ko-sroberta-multitask                           | 0.32     |
| namban4123/Deberta_for_multiple_choice, sentence-transformers/stsb-roberta-base               | 0.30     |
| namban4123/Deberta_for_multiple_choice, sentence-transformers/bert-base-nli-mean-tokens       | 0.33     |
| Riiid/kda-mpnet-base-race, sentence-transformers/all-mpnet-base-v2                            | 0.39     |
| Riiid/kda-mpnet-base-race, sentence-transformers/all-MiniLM-L6-v2                             | 0.39     |
| Riiid/kda-mpnet-base-race, sentence-transformers/msmarco-MiniLM-L-12-v3                       | 0.39     |
| Riiid/kda-mpnet-base-race, jhgan/ko-sroberta-multitask                                        | 0.40     |
| Riiid/kda-mpnet-base-race, sentence-transformers/stsb-roberta-base                            | 0.39     |
| Riiid/kda-mpnet-base-race, sentence-transformers/bert-base-nli-mean-tokens                    | 0.41     |
| LIAMF-USP/aristo-roberta, sentence-transformers/all-mpnet-base-v2                             | 0.38     |
| LIAMF-USP/aristo-roberta, sentence-transformers/all-MiniLM-L6-v2                              | 0.38     |
| LIAMF-USP/aristo-roberta, sentence-transformers/msmarco-MiniLM-L-12-v3                        | 0.38     |
| LIAMF-USP/aristo-roberta, jhgan/ko-sroberta-multitask                                         | 0.39     |
| LIAMF-USP/aristo-roberta, sentence-transformers/stsb-roberta-base                             | 0.38     |
| LIAMF-USP/aristo-roberta, sentence-transformers/bert-base-nli-mean-tokens                     | 0.40     |


## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.



In [ ]:
# insert your code here (load dataset)
from datasets import load_dataset
dataset = load_dataset("winogrande","winogrande_xs")
dataset=dataset['validation'].select(range(100))


/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/winogrande
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/160 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

In [ ]:
# insert your code here (load models)
model_name=["LIAMF-USP/roberta-large-finetuned-race",
            'namban4123/Deberta_for_multiple_choice',
            'LIAMF-USP/aristo-roberta'
]

In [ ]:
# insert your code here (function for predicting best fill)
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch

def func(dataset,model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMultipleChoice.from_pretrained(model_name)
    sum=0
    for example in dataset:
        sentence=example['sentence']
        split_point = sentence.index('_')
        first_sentence = sentence[:split_point].strip()

        second_sentence = sentence[split_point+1:].strip()

        choice1=example['option1']
        choice2=example['option2']
        label=int(example['answer'])-1
        filled_sentences=[choice1+' '+second_sentence,choice2+' '+second_sentence]


        labels = torch.tensor(label).unsqueeze(0)
        labels = torch.tensor([label])
        inputs = tokenizer([first_sentence,first_sentence],filled_sentences, return_tensors="pt", padding=True)
        outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()},labels=labels)
        logits = outputs.logits
        predicted_class = logits.argmax().item()
        if predicted_class==label:
            sum+=1
    return sum/100


Μετατρέψαμε το πρόβλημα σε πρόβλημα multiple choice και χρησιμοποιήσαμε τα παραπάνω μοντέλα του ερωτήματος Β1. Παρακάτω φαίνονται τα 3 καλύτερα μοντέλα βάση του accuracy με καλύτερο όπως και στο Β1 ερώτημα το namban4123/Deberta_for_multiple_choice has accuracy 0.73.

In [ ]:
for model in model_name:
    acc=func(dataset,model)
    print(f'{model} has accuracy {acc}')

LIAMF-USP/roberta-large-finetuned-race has accuracy 0.58
namban4123/Deberta_for_multiple_choice has accuracy 0.73
LIAMF-USP/aristo-roberta has accuracy 0.59


| Model Name                                   | Accuracy |
|----------------------------------------------|----------|
| LIAMF-USP/roberta-large-finetuned-race       | 0.58     |
| namban4123/Deberta_for_multiple_choice       | 0.73     |
| LIAMF-USP/aristo-roberta                     | 0.59     |
